# Сбор данных

In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from urllib.parse import urlencode
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
from collections import defaultdict as dd
import re

In [68]:
# мой proxy scrapper

API_KEY = '*************************'
def get_scrape(url):
    payload = {'api_key': API_KEY, 'url': url}
    proxy_url = 'https://proxy.scrapeops.io/v1/?' + urlencode(payload)
    return proxy_url

#### Буду искать квартиры в новостойках, в пределах Старой Москвы, любое количество комнат, до метро: "пешком не более 45 минут", это необходимо, так как для одной и той же квартиры может показываться расстояние до метро как пешком, так и на машине, так и на общественном транспорте, а вычленять всё это очень неудобно при парсинге. Тем более в большинстве случаев людей интересует всё-таки именно то, как быстро можно добраться до метрополитена на своих двоих. Также установим минимальный лимит на потолки - 2.5 метров, чтобы исключить вероятность того, что на карточке с квартирой указания на высоту потолков будет отсутствовать. Будем искать квартиры со всеми видами отделок.

## Проход по страницам объектов

In [12]:
url = 'https://www.cian.ru/sale/flat/284003619/'

response_object = requests.get(get_scrape(url))
tree_object = BeautifulSoup(response_object.content, 'html.parser')

In [265]:
# достаём цену

price = tree_object.find('div', class_='a10a3f92e9--amount--ON6i1')
span = price.find('span', class_='a10a3f92e9--color_black_100--kPHhJ')
price = int(span.text.replace('\xa0', '').replace('₽', ''))
price

33471000

In [2]:
# достанем часть информации о квартире, которая хранится в одинаковых тэгах - сначала мне было не очень было понятно, как это
# сделать, так как мне постоянно выводило данные для площади. Я обратился к ChatGPT с подводкой:
# "в коде HTML-страницы есть два одинаковых тега с одинаковыми классами, но содержащий разный text как мне в таком случае вытащить эти тексты?"
# Работающий результат получил не сразу - надо использовать select, а не find_all (скрины будут в отдельной папке)

tags = tree_object.select('div.a10a3f92e9--item--Jp5Qv')
features = []
for tag in tags:
    string = tag.text
    features.append(string)
if len(features) == 7:
    features.pop(2)

    
# находим общую площадь квартиры
square = features[0]
square = square.replace('\xa0', '').replace('м²', '').replace(',','.')
square = re.search(r'([\d.]+)', square)
square = square.group(1) if square else ''
print(square)

#находим жилую площадь квартиры 
if len(features) == 6:
    s = features[1]
    area_start = s.find('жилая площадь') + len('жилая площадь')
    area_end = s.find('\xa0', area_start)
    area = s[area_start:area_end].replace(',', '.').strip('ь')
    print(area)

#находим этаж, на котором располагается квартира
if len(features) == 6:
    floor = features[2]
    floor = re.search(r'([\d.]+)', floor)
    floor = floor.group(1) if floor else ''
    print(floor)
else:
    floor = features[1]
    floor = re.search(r'([\d.]+)', floor)
    floor = floor.group(1) if floor else ''
    print(floor)

#находим этажность здания
if len(features) == 6:
    height = features[2]
    height = re.search(r'из (\d+)', height)
    height = height.group(1) if height else ''
    print(height)
else:
    height = features[1]
    height = re.search(r'из (\d+)', height)
    height = height.group(1) if height else ''
    print(height)

#находим год сдачи
if len(features) == 6:
    year = features[3]
    year = year[len('Год сдачи'):]
    year = int(year)
    print(year)
else:
    year = features[2]
    year = year[len('Год сдачи'):]
    year = int(year)
    print(year)

# есть отделка или нет?
if len(features) == 6:
    finishing = features[5]
    finishing = finishing[len('Отделка'):]
    print(finishing)
else:
    finishing = features[4]
    finishing = finishing[len('Отделка'):]
    print(finishing)

NameError: name 'tree_object' is not defined

In [297]:
# достанем время пешком до метро в минутах
metro = (tree_object.find('span', class_ = 'a10a3f92e9--underground_time--YvrcI')).text
metro = int(metro.split()[0])
print(metro)

5


In [298]:
# достанем высоту потолков
list_2 = []
tags_2 = tree_object.select('div.a10a3f92e9--group--K5ZqN')
for tag_2 in tags_2:
    string = tag_2.text
    list_2.append(string)
for item in list_2:
    if 'Высота потолков' in item:
        match = re.search(r'Высота потолков([\d.,]+)', item)
        if match:
            ceiling = match.group(1)
ceiling = ceiling.replace(',', '.')
print(ceiling)

3.3


In [24]:
# достанем район, в котором расположена квартира
list_3 = []
tags_3 = tree_object.select('div.a10a3f92e9--address-line--GRDTb')
for tag_3 in tags_3:
    string = tag_3.text
    list_3.append(string)
    
pattern = r'р-н\s([^,]+)'
matches = re.findall(pattern, string)
if matches:
    district = matches[0]
    print(district)  # Вывод: Левобережный

Левобережный


In [30]:
# собираем всё в кучу

def parse_one_flat(tree_object):
    
    price = tree_object.find('div', class_='a10a3f92e9--amount--ON6i1')
    span = price.find('span', class_='a10a3f92e9--color_black_100--kPHhJ')
    price = int(span.text.replace('\xa0', '').replace('₽', ''))
    
    list_3 = []
    tags_3 = tree_object.select('div.a10a3f92e9--address-line--GRDTb')
    for tag_3 in tags_3:
        string = tag_3.text
        list_3.append(string)
    pattern = r'р-н\s([^,]+)'
    matches = re.findall(pattern, string)
    if matches:
        district = matches[0]
    
    tags = tree_object.select('div.a10a3f92e9--item--Jp5Qv')
    features = []
    for tag in tags:
        string = tag.text
        features.append(string)
    if len(features) == 7:
        features.pop(2)
        
    square = features[0]
    square = square.replace('\xa0', '').replace('м²', '').replace(',','.')
    square = re.search(r'([\d.]+)', square)
    square = square.group(1) if square else ''
    
    if len(features) == 6:
        living_square = features[1]
        area_start = living_square.find('жилая площадь') + len('жилая площадь')
        area_end = living_square.find('\xa0', area_start)
        living_square = living_square[area_start:area_end].replace(',', '.').strip('ь')
    else:
        living_square = 'empty'

    if len(features) == 6:
        floor = features[2]
        floor = re.search(r'([\d.]+)', floor)
        floor = floor.group(1) if floor else ''
    else:
        floor = features[1]
        floor = re.search(r'([\d.]+)', floor)
        floor = floor.group(1) if floor else ''

    if len(features) == 6:
        height = features[2]
        height = re.search(r'из (\d+)', height)
        height = height.group(1) if height else ''
    else:
        height = features[1]
        height = re.search(r'из (\d+)', height)
        height = height.group(1) if height else ''

    if len(features) == 6:
        year = features[3]
        year = year[len('Год сдачи'):]
        year = int(year)
    else:
        year = features[2]
        year = year[len('Год сдачи'):]
        year = int(year)

    if len(features) == 6:
        finishing = features[5]
        finishing = finishing[len('Отделка'):]
    else:
        finishing = features[4]
        finishing = finishing[len('Отделка'):]
        
    metro = (tree_object.find('span', class_ = 'a10a3f92e9--underground_time--YvrcI')).text
    metro = int(metro.split()[0])
    
    list_2 = []
    tags_2 = tree_object.select('div.a10a3f92e9--group--K5ZqN')
    for tag_2 in tags_2:
        string = tag_2.text
        list_2.append(string)
    for item in list_2:
        if 'Высота потолков' in item:
            match = re.search(r'Высота потолков([\d.,]+)', item)
        if match:
            ceiling = match.group(1)
    ceiling = ceiling.replace(',', '.')
    
    info = {}

    info['Цена квартиры'] = price
    info['Район'] = district
    info['Время до метро (пешком)'] = metro
    info['Общая площадь'] = square
    info['Жилая площадь'] = living_square
    info['Этаж'] = floor
    info['Этажей в доме'] = height
    info['Год сдачи'] = year
    info['Отделка'] = finishing
    info['Высота потолков'] = ceiling
    
    return info

In [33]:
#соберём ссылки с одной страницы

url_page = 'https://www.cian.ru/cat.php?deal_type=sale&decorations_list%5B0%5D=fine&decorations_list%5B1%5D=fineWithFurniture&decorations_list%5B2%5D=preFine&decorations_list%5B3%5D=without&engine_version=2&foot_min=45&min_ceiling_height=2.5&minlarea=1&object_type%5B0%5D=2&offer_type=flat&only_foot=2&p=i&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
response = requests.get(get_scrape(url_page))
tree = BeautifulSoup(response.content, 'html.parser')

d = dd(lambda:{})
href_list = []
flats_on_page = tree.find_all('a', class_='_93444fe79c--link--eoxce')
for flat in flats_on_page:
    href = flat.get('href')
    d[href] = {'href': href}

<Response [200]>


In [52]:
d = dd(lambda:{})
for page in range(1,40):
    url_page = f'https://www.cian.ru/cat.php?deal_type=sale&decorations_list%5B0%5D=fine&decorations_list%5B1%5D=fineWithFurniture&decorations_list%5B2%5D=preFine&decorations_list%5B3%5D=without&engine_version=2&foot_min=45&min_ceiling_height=2.5&minlarea=1&object_type%5B0%5D=2&offer_type=flat&only_foot=2&p={page}&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
    response = requests.get(get_scrape(url_page))
    tree = BeautifulSoup(response.content, 'html.parser')
    flats_on_page = tree.find_all('a', class_='_93444fe79c--link--eoxce')
    for flat in flats_on_page:
        href = flat.get('href')
        d[href] = {'href': href}

In [58]:
for i in tqdm(d):
    try:
        response_object = requests.get(get_scrape(i))
        response_object.raise_for_status()  # Генерирует исключение, если получен ответ с ошибкой (не 200 OK)
        tree_object = BeautifulSoup(response_object.content, 'html.parser')
        result = parse_one_flat(tree_object)
        d[i].update(result)
    except requests.exceptions.HTTPError:
        print(f"Ошибка HTTP 500 при запросе {i}. Повторный запрос.")
        # Повторный запрос
        response_object = requests.get(get_scrape(i))
        response_object.raise_for_status()
        tree_object = BeautifulSoup(response_object.content, 'html.parser')
        result = parse_one_flat(tree_object)
        d[i].update(result)
    except Exception as e:
        print(f"Произошла ошибка при запросе {i}: {str(e)}")


  0%|          | 0/1064 [00:00<?, ?it/s]

Ошибка HTTP 500 при запросе https://www.cian.ru/sale/flat/271462103/. Повторный запрос.
Произошла ошибка при запросе https://www.cian.ru/sale/flat/273051726/: local variable 'district' referenced before assignment
Произошла ошибка при запросе https://www.cian.ru/sale/flat/273053307/: local variable 'district' referenced before assignment
Произошла ошибка при запросе https://www.cian.ru/sale/flat/278068554/: local variable 'district' referenced before assignment
Произошла ошибка при запросе https://www.cian.ru/sale/flat/273049981/: local variable 'district' referenced before assignment
Произошла ошибка при запросе https://www.cian.ru/sale/flat/279343463/: local variable 'district' referenced before assignment
Произошла ошибка при запросе https://www.cian.ru/sale/flat/282405966/: local variable 'district' referenced before assignment
Произошла ошибка при запросе https://www.cian.ru/sale/flat/276147600/: 'NoneType' object has no attribute 'find'
Произошла ошибка при запросе https://www.ci

UnboundLocalError: local variable 'district' referenced before assignment

In [61]:
df = pd.DataFrame(d).T
df.to_excel('project.xlsx', index=False)

In [67]:
df['Цена квартиры'].nunique()

738

In [84]:
# Добъём до 1000 объектов
d2 = dd(lambda:{})
for page in range(40,52):
    url_page = f'https://www.cian.ru/cat.php?deal_type=sale&decorations_list%5B0%5D=fine&decorations_list%5B1%5D=fineWithFurniture&decorations_list%5B2%5D=preFine&decorations_list%5B3%5D=without&engine_version=2&foot_min=45&min_ceiling_height=2.5&minlarea=1&object_type%5B0%5D=2&offer_type=flat&only_foot=2&p={page}&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
    response = requests.get(get_scrape(url_page))
    tree = BeautifulSoup(response.content, 'html.parser')
    flats_on_page = tree.find_all('a', class_='_93444fe79c--link--eoxce')
    for flat in flats_on_page:
        href = flat.get('href')
        d2[href] = {'href': href}

In [85]:
for i in tqdm(d2):
    try:
        response_object = requests.get(get_scrape(i))
        response_object.raise_for_status()  # Генерирует исключение, если получен ответ с ошибкой (не 200 OK)
        tree_object = BeautifulSoup(response_object.content, 'html.parser')
        result = parse_one_flat(tree_object)
        d[i].update(result)
    except requests.exceptions.HTTPError:
        print(f"Ошибка HTTP 500 при запросе {i}. Повторный запрос.")
        # Повторный запрос
        response_object = requests.get(get_scrape(i))
        response_object.raise_for_status()
        tree_object = BeautifulSoup(response_object.content, 'html.parser')
        result = parse_one_flat(tree_object)
        d[i].update(result)
    except Exception as e:
        print(f"Произошла ошибка при запросе {i}: {str(e)}")

  0%|          | 0/303 [00:00<?, ?it/s]

Произошла ошибка при запросе https://www.cian.ru/sale/flat/282917838/: 'NoneType' object has no attribute 'find'
Произошла ошибка при запросе https://www.cian.ru/sale/flat/268961290/: 'NoneType' object has no attribute 'find'
Произошла ошибка при запросе https://www.cian.ru/sale/flat/277811300/: local variable 'district' referenced before assignment
Произошла ошибка при запросе https://www.cian.ru/sale/flat/277811262/: local variable 'district' referenced before assignment
Произошла ошибка при запросе https://www.cian.ru/sale/flat/285027301/: 'NoneType' object has no attribute 'find'
Произошла ошибка при запросе https://www.cian.ru/sale/flat/274850345/: local variable 'district' referenced before assignment
Произошла ошибка при запросе https://www.cian.ru/sale/flat/274850349/: local variable 'district' referenced before assignment
Произошла ошибка при запросе https://www.cian.ru/sale/flat/277903588/: local variable 'district' referenced before assignment
Произошла ошибка при запросе ht

In [125]:
df_2 = pd.DataFrame(d).T

In [135]:
df = pd.read_excel('C:/project_one.xlsx')

In [137]:
df = df = df.set_index('href')

In [139]:
df = df.drop('Unnamed: 0', axis=1)

In [148]:
result = pd.concat([df, df_2])
result.to_excel('MaximDavydovProject.xlsx')